<a href="https://colab.research.google.com/github/BiggusMaximus/Tensorflow_Certification_Preparation_Dicoding/blob/main/A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow==2.7.0
!pip install pandas==1.3.4
!pip install PIL==8.4.0
!pip install tensorflow_datasets==4.4.0
!pip install numpy==1.21.4
!pip install scipy==1.7.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 665.5 MB 22 kB/s 
     |████████████████████████████████| 463 kB 34.9 MB/s 
     |████████████████████████████████| 1.3 MB 61.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.2+zzzcolab20220527125636
    Uninstalling tensorflow-2.8.2+zzzcolab20220527125636:
      Successfully uninstalled tensorflow-2.8.2+zzzcolab20220

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 38.1 MB 415 kB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [3]:
import tensorflow as tf
import pandas as pd
import PIL
import tensorflow_datasets
import numpy as np
import scipy as sp
import platform

In [4]:
print(f"Python version : {platform.python_version()}")
print(f"tensorflow version : {tf.__version__}")
print(f"pandas version : {pd.__version__}")
print(f"pillow version : {PIL.__version__}")
print(f"tensorflow_datasets version : {tensorflow_datasets.__version__}")
print(f"numpy version : {np.version.version}")
print(f"scipy version : {sp.__version__}")


Python version : 3.7.13
tensorflow version : 2.7.0
pandas version : 1.3.4
pillow version : 7.1.2
tensorflow_datasets version : 4.4.0
numpy version : 1.21.6
scipy version : 1.7.3


In [15]:
# =====================================================================================
# PROBLEM A2
#
# Build a Neural Network Model for Horse or Human Dataset.
# The test will expect it to classify binary classes.
# Your input layer should accept 150x150 with 3 bytes color as the input shape.
# Don't use lambda layers in your model.
#
# The dataset used in this problem is created by Laurence Moroney (laurencemoroney.com).
#
# Desired accuracy and validation_accuracy > 83%
# ======================================================================================

import tensorflow as tf
import zipfile
import urllib.request
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def solution_A2():
    data_url_1 = 'https://github.com/dicodingacademy/assets/releases/download/release-horse-or-human/horse-or-human.zip'
    urllib.request.urlretrieve(data_url_1, 'horse-or-human.zip')
    local_file = 'horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/horse-or-human')

    data_url_2 = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/validation-horse-or-human.zip'
    urllib.request.urlretrieve(data_url_2, 'validation-horse-or-human.zip')
    local_file = 'validation-horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/validation-horse-or-human')
    zip_ref.close()

    model = tf.keras.models.Sequential([

        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(300, 300, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=2e-4,                                                         
        decay_steps=200,
        decay_rate=0.90
    )
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=lr_schedule
    )

    model.compile(loss='binary_crossentropy', 
                  optimizer=optimizer, 
                  metrics=['accuracy'])

    TRAINING_DIR = 'data/horse-or-human'
    train_datagen = ImageDataGenerator(rescale=1. / 255)
    # YOUR IMAGE SIZE SHOULD BE 150x150
    train_generator = train_datagen.flow_from_directory(
        TRAINING_DIR,
        target_size=(300, 300),
        batch_size=20,
        class_mode='binary')

    VALIDATION_DIR = 'data/validation-horse-or-human'
    validation_datagen = ImageDataGenerator(rescale=1 / 255)
    validation_generator = validation_datagen.flow_from_directory(
        VALIDATION_DIR,
        target_size=(300, 300),
        batch_size=20,
        class_mode='binary')



    EPOCHS = 20

    model.fit(
        train_generator,
        epochs=EPOCHS,
        steps_per_epoch = 50,
        verbose=1,
        validation_data=validation_generator,
        validation_steps = 12,
    )
    return model


# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model=solution_A2()
    model.save("model_A2.h5")


Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
Epoch 1/20
50/50 [==============================] - 9s 170ms/step - loss: 0.5015 - accuracy: 0.7345 - val_loss: 0.8649 - val_accuracy: 0.8375
Epoch 2/20
50/50 [==============================] - 8s 167ms/step - loss: 0.1091 - accuracy: 0.9605 - val_loss: 1.4036 - val_accuracy: 0.8208
Epoch 3/20
50/50 [==============================] - 8s 163ms/step - loss: 0.0623 - accuracy: 0.9757 - val_loss: 2.2248 - val_accuracy: 0.7792
Epoch 4/20
50/50 [==============================] - 8s 165ms/step - loss: 0.0442 - accuracy: 0.9787 - val_loss: 1.1343 - val_accuracy: 0.8542
Epoch 5/20
50/50 [==============================] - 8s 163ms/step - loss: 0.0082 - accuracy: 0.9990 - val_loss: 1.4793 - val_accuracy: 0.8375
Epoch 6/20
50/50 [==============================] - 8s 163ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 1.5520 - val_accuracy: 0.8625
Epoch 7/20
50/50 [==============================] - 8s 164ms/step